# 电子积分准备

同[非周期性教程](../qcbasic/basis_integral.ipynb)一样，我们先来准备一个尽可能小的体系。此处选择的是金刚石的最小重复单元，含两个碳原子。由于使用了GTH赝势，只有8个电子。

不过周期性的核排斥能量的计算没有非周期性那么简单，因此我们先来看电子积分。

In [1]:
from pyscf.pbc import gto

cell = gto.Cell()
cell.atom='''
C 0.000000000000   0.000000000000   0.000000000000
C 1.685068664391   1.685068664391   1.685068664391
'''
cell.a = '''
0.000000000, 3.370137329, 3.370137329
3.370137329, 0.000000000, 3.370137329
3.370137329, 3.370137329, 0.000000000'''
cell.unit = 'B'
cell.basis = 'gth-szv'
cell.pseudo = 'gth-pade'
cell.build()

In [2]:
cell.nelec

(4, 4)

GTH-SZV 是一个足够小的基组，对于目前体系只有8个基函数。

In [3]:
cell._basis

{'C': [[0,
   [4.3362376436, 0.1490797872],
   [1.2881838513, -0.0292640031],
   [0.4037767149, -0.688204051],
   [0.1187877657, -0.3964426906]],
  [1,
   [4.3362376436, -0.0878123619],
   [1.2881838513, -0.27755603],
   [0.4037767149, -0.4712295093],
   [0.1187877657, -0.4058039291]]]}

In [4]:
cell.nao

8

## 重叠积分

首先，即使不了解重叠积分的本质，我们也可以找到这样的API来计算周期性体系的重叠积分

In [5]:
S_ref = cell.pbc_intor('int1e_ovlp')

In [6]:
S_ref.shape

(8, 8)

然而，这样的重叠积分与不存在周期性边界条件时的重叠积分是不一样的（习题：尝试验证这一点）。这是因为周期性情形下的AO实际上是crystalline cGTO

相关定义

* `ao` crystalline AO $\phi$
* （非周期性的）AO $\mu$
* 晶格偏移矢量 $\boldsymbol L$

值得注意的是，在非周期性笔记中，我们只有一种AO，用 $\phi$ 表示。而现在 $\phi$ 用于表示crystalline AO，原来的AO改用$\mu$表示。

计算crystaline AO的格点表示，只需把AO及邻近镜像的格点表示都加起来
$$ \phi(\boldsymbol{r}) = \dfrac{1}{\sqrt{N}} \sum_L \mu(\boldsymbol{r} - \boldsymbol{L}) $$

当然，在此之前，我们需要先生成一套格点

In [7]:
mesh = [10,10,10]
grids = cell.get_uniform_grids(mesh)

In [8]:
grids.shape

(1000, 3)

下面就可以生成邻近镜像的偏移矢量，然后计算crystaline AO的格点表示。（习题：`n_images = [2,2,2]`实际上包含多少个镜像？）

In [9]:
import numpy as np
def get_ao_values(cell, n_images, grids):
    cell = cell.copy()
    Ls = get_lattice_Ls(cell, n_images)
    atom_ref = cell.atom_coords()

    ao = 0
    natm = cell.natm
    for L in Ls:
        cell.atom = [[cell._atom[i][0], atom_ref[i] + L] for i in range(natm)]
        cell.build()
        ao += cell.eval_gto('GTOval', grids)
    return ao

def get_lattice_Ls(cell, n_images):
    n_images_x = n_images[0]
    n_images_y = n_images[1]
    n_images_z = n_images[2]
    Ls = []
    for ix in range(-n_images_x, n_images_x+1):
        for iy in range(-n_images_y, n_images_y+1):
            for iz in range(-n_images_z, n_images_z+1):
                L = np.einsum('x,xy->y', np.array((ix, iy, iz)), cell.lattice_vectors())
                Ls.append(L)
    return Ls

n_images = [2,2,2]
ao = get_ao_values(cell, n_images, grids)


可以与PySCF的标准实现比较一下。`n_images`取得越大，就越接近精确值。（TODO：如何估计合适的`n_images`？）

In [10]:
ao_ref = cell.pbc_eval_gto('GTOval', grids)
print(abs(ao-ao_ref).max())

0.00018057172454068182


下面计算重叠积分。通过上面对crystalline AO的了解，至少有两种办法，一种是直接在格点上求和

In [11]:
def get_ovlp1(cell, n_images, grids):
    weight = cell.vol / np.prod(mesh)
    ao = get_ao_values(cell, n_images, grids)
    s = weight * np.einsum('xi,xj->ij', ao.conj(), ao)
    return s

S1 = get_ovlp1(cell, n_images, grids)
print(abs(S1 - S_ref).max())

0.0004307987275599112


另一种是对晶格偏移展开，

$$
\langle \phi_\mu | \phi_\nu \rangle = \dfrac{1}{N} \sum_{\boldsymbol{L L}'} \int \mathrm{d} r \mu(\boldsymbol{r} - \boldsymbol{L})^* \nu(\boldsymbol{r} - \boldsymbol{L}') \\
= \dfrac{1}{N} \sum_{\boldsymbol{L}} \int \mathrm{d} r \mu(\boldsymbol{r})^* \nu(\boldsymbol{r} + \boldsymbol{L}) 
$$

In [12]:
from pyscf import gto as mole_gto
def get_ovlp2(cell, n_images):
    cellL = cell.copy()
    Ls = get_lattice_Ls(cell, n_images)
    atom_ref = cell.atom_coords()

    s = 0
    natm = cell.natm
    for L in Ls:
        cellL.atom = [[cell._atom[i][0], atom_ref[i] + L] for i in range(natm)]
        cellL.build()
        s += mole_gto.intor_cross('int1e_ovlp', cell, cellL)
    return s

S2 = get_ovlp2(cell, n_images)
print(abs(S2 - S_ref).max())

0.007028588146315767


由于之前所取的`n_images`比较小，这里的误差稍大了一些。我们增大`n_images`试试看。

In [13]:
S2 = get_ovlp2(cell, [3,3,3])
print(abs(S2 - S_ref).max())

3.5862632974045794e-05


## 动能积分

我们尝试用类似的方法计算动能积分。sunqm 的原教程只演示了格点方式，我们来尝试第二种方法(lattice sum)。

In [15]:
kin_ref = cell.pbc_intor('int1e_kin')

def get_kin2(cell, n_images):
    cellL = cell.copy()
    Ls = get_lattice_Ls(cell, n_images)
    atom_ref = cell.atom_coords()

    s = 0
    natm = cell.natm
    for L in Ls:
        cellL.atom = [[cell._atom[i][0], atom_ref[i] + L] for i in range(natm)]
        cellL.build()
        s += mole_gto.intor_cross('int1e_kin', cell, cellL)
    return s

kin2 = get_kin2(cell, n_images)
print(abs(kin2 - kin_ref).max())

kin2 = get_kin2(cell, [3,3,3])
print(abs(kin2 - kin_ref).max())

0.003957462046596236
4.369938133889917e-05


看起来也是可行的。

注：我们所实现的以上两种积分的方法二，实际上和作为参照标准的`cell.pbc_intor()`是同一种方法，只是后者实现更复杂，效率更高。这也是PySCF目前默认的计算这两种积分的方式，所以在GTO基下应该是效率最高的方式（不过我们还没有证明这一点）。

## 电子-核吸引能积分

此种积分看起来也可以用上面所说的两种办法来求。但是，不同于上面的两种两中心的积分，我们还得考虑原子核位置的周期性。这样 lattice sum 时就多了一重求和（这是唯一的问题吗？），看起来不太方便。我们可以考虑用傅里叶变换来简化计算。且看下一节。